# 📔 Jupyter Notebook: RAG Avanzado con PDFs y Chunking

## Objetivo de la Clase

En esta versión del notebook, damos un paso más allá. En lugar de usar fragmentos de código pre-definidos, vamos a construir un sistema RAG que consume un **documento PDF** como fuente de conocimiento. Esto nos obligará a implementar uno de los pasos más cruciales del proceso: el **Chunking (fragmentación)**.

Aprenderemos a:
1. Extraer texto de un archivo PDF.
2. Dividir un texto largo en fragmentos (chunks) de un tamaño fijo.
3. Utilizar estos chunks para construir la base de datos vectorial y responder preguntas sobre el contenido del documento.

### **Paso 0: Instalación y Configuración de Dependencias**

Además de las librerías anteriores, ahora necesitamos una para leer archivos PDF. Usaremos `PyMuPDF`, que es rápida y eficiente.

In [ ]:
# Descomenta y ejecuta la siguiente línea si no tienes instaladas las librerías
# !pip install openai google-generativeai cohere numpy scikit-learn PyMuPDF

In [ ]:
import os
import sqlite3
import numpy as np
import json
import openai
import google.generativeai as genai
import cohere
import fitz  # PyMuPDF

# --- CONFIGURACIÓN DE LAS API KEYS ---
# Asegúrate de tener tus claves de API como variables de entorno
# o reemplaza el placeholder con tu clave correspondiente.
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ["GEMINI_API_KEY"] = "YOUR_GEMINI_API_KEY"
os.environ["COHERE_API_KEY"] = "YOUR_COHERE_API_KEY"

openai.api_key = os.getenv("OPENAI_API_KEY")
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
co = cohere.Client(os.getenv("COHERE_API_KEY"))

print("Librerías importadas y APIs configuradas.")

### **Paso 1: Preparar la Base de Conocimiento (El PDF)**

Primero, necesitamos nuestro documento. Para que este notebook sea autocontenido, crearemos un PDF de ejemplo con un texto largo sobre la historia de la IA. Luego, implementaremos una función para extraer todo el texto de ese PDF.

In [ ]:
def crear_pdf_de_ejemplo(nombre_archivo, texto):
    """Crea un archivo PDF simple con el texto proporcionado."""
    doc = fitz.open() # Nuevo documento PDF
    pagina = doc.new_page()
    # Inserta el texto. 'rect' define el área donde se insertará el texto.
    rect = fitz.Rect(50, 50, 550, 800) # x0, y0, x1, y1
    pagina.insert_text(texto, rect, fontsize=12, fontname="helv", rotate=0)
    doc.save(nombre_archivo)
    doc.close()
    print(f"PDF de ejemplo '{nombre_archivo}' creado con éxito.")

def extraer_texto_de_pdf(ruta_pdf):
    """Extrae el texto completo de un archivo PDF."""
    doc = fitz.open(ruta_pdf)
    texto_completo = ""
    for pagina in doc:
        texto_completo += pagina.get_text()
    doc.close()
    return texto_completo

# Contenido para nuestro PDF de ejemplo
texto_ia = """
Historia de la Inteligencia Artificial

La historia de la inteligencia artificial (IA) es fascinante y se remonta a la antigüedad, con mitos e historias sobre seres artificiales dotados de inteligencia. Sin embargo, el campo moderno de la IA no comenzó a tomar forma hasta mediados del siglo XX, impulsado por avances en la computación.

La conferencia de Dartmouth en 1956 es ampliamente considerada como el evento que acuñó el término "inteligencia artificial" y lanzó el campo como un área formal de investigación. John McCarthy, Marvin Minsky, Nathaniel Rochester y Claude Shannon organizaron este taller con el objetivo de explorar la conjetura de que cada aspecto del aprendizaje o cualquier otra característica de la inteligencia puede, en principio, ser descrito con tanta precisión que se puede hacer que una máquina lo simule. Este evento marcó el inicio de décadas de investigación, caracterizadas por olas de optimismo y períodos de "invierno de la IA", donde la financiación y el interés disminuyeron.

Uno de los pioneros más importantes fue Alan Turing, un matemático y lógico británico. En su artículo de 1950, "Computing Machinery and Intelligence", Turing propuso lo que ahora se conoce como la Prueba de Turing. Esta prueba evalúa la capacidad de una máquina para exhibir un comportamiento inteligente indistinguible del de un ser humano. Turing no solo sentó las bases teóricas de la computación con su concepto de la Máquina de Turing, sino que también planteó preguntas filosóficas profundas sobre la naturaleza de la mente y la inteligencia que siguen siendo relevantes hoy en día. Su trabajo fue fundamental para el desarrollo de la informática y la IA.

Durante los años 60 y 70, la investigación se centró en la resolución de problemas y los métodos simbólicos. Programas como el "General Problem Solver" de Newell y Simon intentaron imitar el pensamiento humano para resolver problemas lógicos. Sin embargo, estas primeras aproximaciones tropezaron con la "explosión combinatoria": a medida que los problemas se volvían más complejos, la cantidad de posibles soluciones a explorar crecía exponencialmente, haciendo que los cálculos fueran inviables.

El resurgimiento de la IA en los años 80 vino de la mano de los sistemas expertos, programas diseñados para emular la capacidad de toma de decisiones de un experto humano en un dominio específico. Aunque tuvieron éxito comercial, su creación era costosa y su conocimiento, frágil y difícil de mantener.

El verdadero cambio de paradigma llegó con el auge del aprendizaje automático (machine learning) en los años 90 y 2000, y más específicamente, con el aprendizaje profundo (deep learning) a partir de 2010. En lugar de programar reglas explícitas, los modelos de aprendizaje automático aprenden patrones directamente de los datos. Geoffrey Hinton, Yann LeCun y Yoshua Bengio, a menudo llamados los "padrinos de la IA", fueron pioneros en el desarrollo de redes neuronales profundas. Sus contribuciones en áreas como las redes neuronales convolucionales (CNN) para la visión por computadora y las redes neuronales recurrentes (RNN) para el procesamiento del lenguaje natural revolucionaron el campo. El avance en la capacidad de cómputo, especialmente con las GPUs, y la disponibilidad de grandes conjuntos de datos (Big Data) fueron los catalizadores que permitieron que el deep learning floreciera.

Hoy, la IA está en todas partes, desde los asistentes de voz en nuestros teléfonos hasta los algoritmos que recomiendan contenido en plataformas de streaming y los modelos de lenguaje grandes como GPT y Gemini, que pueden generar texto coherente y responder preguntas complejas. La investigación continúa a un ritmo acelerado, explorando nuevas arquitecturas, abordando problemas de ética y seguridad, y empujando los límites de lo que las máquinas pueden aprender y hacer.
"""

PDF_FILENAME = "historia_ia.pdf"
crear_pdf_de_ejemplo(PDF_FILENAME, texto_ia)

# Extraer el texto del PDF que acabamos de crear
texto_extraido = extraer_texto_de_pdf(PDF_FILENAME)

print("\n--- Inicio del Texto Extraído del PDF ---")
print(texto_extraido[:500])
print("...")

### **Paso 2: Chunking (Fragmentación) del Texto**

Ahora que tenemos el texto completo, debemos dividirlo en chunks. Una estrategia simple y efectiva es el **chunking de tamaño fijo**. Dividiremos el texto en fragmentos de un número determinado de caracteres.

Elegir el tamaño del chunk es importante:
- **Chunks demasiado grandes:** Pueden contener información irrelevante que diluya el contexto útil.
- **Chunks demasiado pequeños:** Pueden carecer del contexto necesario para que el embedding capture el significado completo.

Para este ejemplo, usaremos un tamaño de **2000 caracteres**, como se solicitó.

In [ ]:
def dividir_en_chunks(texto, chunk_size=2000):
    """Divide un texto en chunks de un tamaño específico."""
    return [texto[i:i + chunk_size] for i in range(0, len(texto), chunk_size)]

CHUNK_SIZE = 2000
text_chunks = dividir_en_chunks(texto_extraido, chunk_size=CHUNK_SIZE)

print(f"El texto ha sido dividido en {len(text_chunks)} chunks.")
print("\n--- Ejemplo del Primer Chunk ---")
print(text_chunks[0])

### **Paso 3: Generación de Embeddings**

Este paso no cambia. Las funciones que creamos para generar embeddings pueden tomar cualquier cadena de texto, así que funcionarán perfectamente con nuestros nuevos chunks.

In [ ]:
# Diccionario para mapear nombres de modelos a sus dimensiones
EMBEDDING_MODELS = {
    "openai": {"model": "text-embedding-3-small", "dimensions": 1536},
    "gemini": {"model": "models/text-embedding-004", "dimensions": 768},
    "cohere": {"model": "embed-multilingual-v3.0", "dimensions": 1024}
}

def get_openai_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    response = openai.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

def get_gemini_embedding(text, model="models/text-embedding-004"):
    return genai.embed_content(model=model, content=text)["embedding"]

def get_cohere_embedding(text, model="embed-multilingual-v3.0"):
    response = co.embed(texts=[text], model=model)
    return response.embeddings[0]

print("Funciones de embedding definidas.")

### **Paso 4: Almacenamiento en SQLite**

Ahora, poblaremos nuestra base de datos SQLite con los chunks de texto del PDF y sus correspondientes embeddings.

In [ ]:
DB_FILE = "rag_database_pdf.db"

def setup_database():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS text_embeddings") # Empezar de cero
    cursor.execute("""
        CREATE TABLE text_embeddings (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            model_name TEXT NOT NULL,
            chunk TEXT NOT NULL,
            embedding BLOB NOT NULL
        )
    """)
    conn.commit()
    conn.close()
    print(f"Base de datos '{DB_FILE}' y tabla 'text_embeddings' creadas.")

def store_embeddings(model_name, chunks, embeddings):
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    for chunk, embedding in zip(chunks, embeddings):
        embedding_blob = np.array(embedding, dtype=np.float32).tobytes()
    cursor.execute(
        "INSERT INTO text_embeddings (model_name, chunk, embedding) VALUES (?, ?, ?)",
        (model_name, chunk, embedding_blob)
    )
    conn.commit()
    conn.close()

# --- Proceso Principal de Población de la Base de Datos ---
setup_database()

chosen_model_name = "gemini" # Puedes cambiar a "openai" o "cohere"
print(f"\nGenerando y almacenando embeddings para {len(text_chunks)} chunks usando el modelo: {chosen_model_name}...")

embedding_function = get_gemini_embedding # Asignamos la función directamente

# Generamos los embeddings para cada chunk de texto
embeddings_to_store = [embedding_function(chunk) for chunk in text_chunks]

# Los almacenamos en la base de datos
store_embeddings(chosen_model_name, text_chunks, embeddings_to_store)

print(f"¡Éxito! Se han almacenado {len(text_chunks)} chunks y sus embeddings en '{DB_FILE}'.")

### **Paso 5: Retrieval (Recuperación de Información)**

La lógica de recuperación es idéntica a la del ejemplo anterior. La función buscará en la nueva tabla y comparará la query del usuario con los embeddings de los chunks del PDF.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_top_k(query, model_name, top_k=2):
    print(f"Buscando los {top_k} chunks más relevantes para la query usando el modelo '{model_name}'...")

    embedding_function = get_gemini_embedding
    query_embedding = embedding_function(query)
    query_embedding_np = np.array(query_embedding).reshape(1, -1)

    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT chunk, embedding FROM text_embeddings WHERE model_name = ?", (model_name,))
    db_results = cursor.fetchall()
    conn.close()

    if not db_results:
        print("No se encontraron embeddings en la base de datos.")
        return []

    db_chunks = [row[0] for row in db_results]
    embedding_dim = EMBEDDING_MODELS[model_name]["dimensions"]
    db_embeddings = [np.frombuffer(row[1], dtype=np.float32) for row in db_results]
    db_embeddings_np = np.array(db_embeddings)

    similarities = cosine_similarity(query_embedding_np, db_embeddings_np)[0]
    top_k_indices = np.argsort(similarities)[::-1][:top_k]
    relevant_chunks = [db_chunks[i] for i in top_k_indices]
    
    print("Chunks recuperados con éxito.")
    return relevant_chunks

# --- Prueba de Recuperación ---
user_query = "¿Quién fue Alan Turing y cuál fue su contribución a la IA?"
retrieved_chunks = retrieve_top_k(user_query, chosen_model_name, top_k=1)

print("\n--- Contexto Recuperado para la pregunta sobre Alan Turing ---")
for i, chunk in enumerate(retrieved_chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")

### **Paso 6: Augmentation y Generation**

Finalmente, usamos los chunks recuperados del PDF para aumentar el prompt y generar una respuesta precisa y contextualizada.

In [ ]:
def build_prompt(query, context_chunks):
    context = "\n\n---\n\n".join(context_chunks)
    prompt = f"""
Eres un asistente experto que responde preguntas sobre un documento. Responde a la pregunta del usuario basándote únicamente en el siguiente contexto extraído del documento.
Si el contexto no contiene la información necesaria para responder, di explícitamente: 'La información no se encuentra en el documento proporcionado'.

Contexto:
{context}

Pregunta del usuario:
{query}

Respuesta:
"""
    return prompt

def get_llm_response(prompt, provider="gemini"):
    print(f"\nEnviando prompt aumentado a {provider.upper()}...")
    try:
        if provider == "openai":
            response = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}]
            )
            return response.choices[0].message.content
        elif provider == "gemini":
            model = genai.GenerativeModel('gemini-1.5-flash')
            response = model.generate_content(prompt)
            return response.text
        elif provider == "cohere":
            response = co.chat(model='command-r', message=prompt)
            return response.text
    except Exception as e:
        return f"Error al contactar la API de {provider}: {e}"

# --- Proceso Final: Poniéndolo Todo Junto ---
final_user_query = "¿Qué fue la conferencia de Dartmouth y por qué es importante para la IA?"
context_chunks = retrieve_top_k(final_user_query, chosen_model_name, top_k=2)
final_prompt = build_prompt(final_user_query, context_chunks)

print("\n--- Prompt Final Enviado al LLM ---")
print(final_prompt)

# --- Generación con Gemini ---
response_gemini = get_llm_response(final_prompt, provider="gemini")
print("\n--- Respuesta de Gemini ---")
print(response_gemini)

# --- Generación con OpenAI ---
response_openai = get_llm_response(final_prompt, provider="openai")
print("\n--- Respuesta de OpenAI ---")
print(response_openai)